In [185]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, GlobalMaxPooling1D, Embedding, Input, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from datetime import datetime
import pickle



2.18.0


In [186]:
#variables
kernel_size=3
hidden_dims=250
test_size = 0.3
batch_size=32
#accuracy gains seem to be minimal after epoch 10
epochs=4
dp = 0.2
lr = 0.001
threshold=0.5
random_seed = 1
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
#dataset used: https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression
#loading data
try:
    data = pd.read_csv("data/Student_Performance.csv")
except Exception as e:
    print(f"Error with getting file: {e}")

print(data.head())

#convery strings/integers to 1 for yes or 0 for no for extracurricular activities and previous scores
binary_extra = [1 if d.lower() == "yes" else 0 for d in data["Extracurricular Activities"]]
binary_pass = [1 if d >= 50.0 else 0 for d in data["Performance Index"]]

#print(data.isnull().sum())
#print(data.shape)

X = data.drop("Performance Index", axis=1)
y = binary_pass

X["Extracurricular Activities"] = binary_extra

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_seed)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape)
print(X_test.shape)



   Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0              7               99                        Yes            9   
1              4               82                         No            4   
2              8               51                        Yes            7   
3              5               52                        Yes            5   
4              7               75                         No            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  
(7000, 5, 1)
(3000, 5, 1)


In [188]:
#define and compile model
model = Sequential([
    Conv1D(64, 2, activation="relu", input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),
    Conv1D(128, 2, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    GlobalMaxPooling1D(),
    Dense(128, activation="relu"),
    Dropout(dp),
    Dense(1, activation="sigmoid")

])

model.compile(optimizer=Adam(learning_rate=lr),
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_46 (Conv1D)              │ (None, 4, 64)          │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_69 (Dropout)            │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_47 (Conv1D)              │ (None, 3, 128)         │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 3, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_70 (Dropout)            │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_16         │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_71 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,113 (133.25 KB)

 Trainable params: 33,729 (131.75 KB)

 Non-trainable params: 384 (1.50 KB)

In [189]:
#train model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/4
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7770 - loss: 0.4623 - val_accuracy: 0.8267 - val_loss: 0.4044
Epoch 2/4
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8660 - loss: 0.2936 - val_accuracy: 0.9363 - val_loss: 0.2048
Epoch 3/4
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8896 - loss: 0.2467 - val_accuracy: 0.9370 - val_loss: 0.1679
Epoch 4/4
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9041 - loss: 0.2260 - val_accuracy: 0.9550 - val_loss: 0.1408


In [190]:
#predictions
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > threshold).astype("int")
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: " + str(accuracy * 100))
print("Precision: " + str(precision * 100))
print("Recall: " + str(recall * 100))
print("F1 Score: " + str(f1 * 100))

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 95.5
Precision: 95.88732394366197
Recall: 96.48526077097506
F1 Score: 96.18536309692003


In [191]:
#save model and scaler
try:
    curr_date = datetime.now().strftime("%Y%m%d_%H%M_%f")

    model.save(f"models/classification_model.h5")
except Exception as e:
    print(f"Error with saving model: {e}")

try:
    with open("scalers/classification_scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)
except Exception as e:
    print(f"Error with saving scaler: {e}")